In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 5.2MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 5.2MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
 SUFFIX_CAT = '_cat'
 for feat in df.columns:
  if isinstance(df[feat][0],list) : continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT]= factorized_values

In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model (model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision Tree

In [0]:
run_model( DecisionTreeRegressor(max_depth=5), cat_feats ) 

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats )

(-18718.657185256638, 64.5424578125788)

## RGBoost

In [0]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0,
    'random_state': 0
}
x = df[feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)
run_model(xgb.XGBRegressor(**xgb_params), feats )

[19:25:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:25:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:26:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(x, y)

imp = PermutationImportance(m, random_state=0).fit(x, y)
eli5.show_weights(imp, feature_names=cat_feats)

[18:02:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd_cat
0.1132 ± 0.0032,param_rok-produkcji_cat
0.1090 ± 0.0025,param_stan_cat
0.0619 ± 0.0025,param_skrzynia-biegów_cat
0.0568 ± 0.0016,param_faktura-vat_cat
0.0489 ± 0.0014,param_moc_cat
0.0273 ± 0.0007,param_marka-pojazdu_cat
0.0242 ± 0.0013,feature_kamera-cofania_cat
0.0212 ± 0.0008,param_typ_cat
0.0174 ± 0.0008,param_pojemność-skokowa_cat


In [0]:
len(cat_feats)

151

In [0]:
'param_napęd_cat'
'param_rok-produkcji_cat'
'param_stan_cat'
'param_skrzynia-biegów_cat'
'param_faktura-vat_cat'
'param_moc_cat'
'param_marka-pojazdu_cat'
'feature_kamera-cofania_cat'
'param_typ_cat'
'param_pojemność-skokowa_cat'
'seller_name_cat'
'feature_wspomaganie-kierownicy_cat'
'param_model-pojazdu_cat'
'param_wersja_cat'
'param_kod-silnika_cat'
'feature_system-start-stop_cat'
'feature_asystent-pasa-ruchu_cat'
'feature_czujniki-parkowania-przednie_cat'
'feature_łopatki-zmiany-biegów_cat'
'feature_regulowane-zawieszenie_cat'

[19:14:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd_cat
0.1132 ± 0.0032,param_rok-produkcji_cat
0.1090 ± 0.0025,param_stan_cat
0.0619 ± 0.0025,param_skrzynia-biegów_cat
0.0568 ± 0.0016,param_faktura-vat_cat
0.0489 ± 0.0014,param_moc_cat
0.0273 ± 0.0007,param_marka-pojazdu_cat
0.0242 ± 0.0013,feature_kamera-cofania_cat
0.0212 ± 0.0008,param_typ_cat
0.0174 ± 0.0008,param_pojemność-skokowa_cat


In [0]:
feats = ['param_napęd_cat','param_rok-produkcji_cat','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:26:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:26:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:26:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc_cat','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:26:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:26:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:26:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [0]:
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0]) )

df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa_cat','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:31:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:31:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:31:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [0]:


df[ 'param_pojemność-skokowa'] = df[ 'param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')) )

feats = ['param_napęd_cat','param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc','param_marka-pojazdu_cat','feature_kamera-cofania_cat','param_typ_cat','param_pojemność-skokowa','seller_name_cat','feature_wspomaganie-kierownicy_cat','param_model-pojazdu_cat','param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat','feature_czujniki-parkowania-przednie_cat','feature_łopatki-zmiany-biegów_cat','feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:37:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:37:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:38:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)